<a href="https://colab.research.google.com/github/sagerenag/Hierarchical_Forecast/blob/main/Hierarchical_Forecast.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Importante

este notebook esta pensado para ejecutarse en google colab, se recomienda tambien tener en su unidad los csv usados en este codigo (ubicados en la carpeta csv de este repo) o por lo menos, unos csv que cumplan con las caracteristicas esperadas en la data para evitar cualquier inconveniente con la ejecucion del mismo

## pipelines

In [ ]:
# Instalar librerias necesarias
!pip install hierarchicalforecast
!pip install ydata-profiling
!pip install statsforecast
!pip install mlforecast neuralforecast
!pip install xgboost
!pip install datasetsforecast
!pip install imbalanced-learn


# Import required libraries
import pandas as pd
import numpy as np
from itertools import product
from imblearn.pipeline import Pipeline


# Import additional libraries
import pandas as pd
from sklearn.preprocessing import  FunctionTransformer
from sklearn import preprocessing as prep
from sklearn.metrics import mean_absolute_error


# Import forecasting libraries
from hierarchicalforecast.core import HierarchicalReconciliation
from hierarchicalforecast.evaluation import HierarchicalEvaluation
from hierarchicalforecast.methods import OptimalCombination, MinTrace, BottomUp
from hierarchicalforecast.utils import aggregate
from statsforecast.core import StatsForecast
from statsforecast.models import DynamicOptimizedTheta as DOT
from mlforecast import MLForecast
from neuralforecast import NeuralForecast
from neuralforecast.auto import AutoLSTM
from xgboost import XGBRegressor
from hyperopt import STATUS_OK, Trials, fmin, hp, tpe
from datasetsforecast.losses import rmse

# cargar los datos en csv
from google.colab import drive
from google.colab import files

# Mount Google Drive
drive.mount('/content/drive')

# Load data from CSV files
productos = pd.read_csv('/content/drive/MyDrive/Tabla_Maestra_Productos.csv')
Ventas = pd.read_csv('/content/drive/MyDrive/Ventas.csv')


# Define a function to aggregate hierarchies in the data
def aggregate_hierarchies(df, column):
    hierarchies = [['Pais'], ['Pais', 'Proyecto'], ['Pais', 'Proyecto', 'SnkProducto']]
    df, H_df, tags = aggregate(
    df[['SnkFecha', 'SnkProducto', column, 'Pais', 'Proyecto']].rename(columns={'SnkFecha':'ds', column:'y'}),
    spec = hierarchies
    )
    df = df.reset_index()
    return(df, H_df, tags)

# Define a function to filter product data
def filter_producto(productos):
    productos = productos[["SnkProducto", 'Codigo']]
    productos['Codigo'] = productos['Codigo'].astype(str)
    # get final products
    filtro = pd.read_excel("/content/drive/MyDrive/filtro.xlsx", header=1)
    filtro = filtro['SAP'].tolist()
    filtro = list(map(str, filtro))
    #apply filter to products
    productos = productos[productos['Codigo'].isin(filtro)].drop(['Codigo'], axis=1)
    return productos

# Define a function to subset sales data
def subset_ventas(df):
    return df[['SnkFecha', 'SnkProducto', 'Unidades', 'Proyecto', 'NumeroDocumento']]

# Define a function to remove duplicates in sales data
def eliminar_duplicados(df):
    return df.drop_duplicates(subset=['NumeroDocumento', 'Unidades', 'SnkProducto'])

# Define a function to select specific columns in sales data
def seleccionar_columnas(df):
    df['SnkProducto'] = df['SnkProducto'].astype(str)
    return df[['SnkFecha', 'SnkProducto', 'Unidades', 'Proyecto']]

# Define a function to filter sales with positive units
def filtrar_ventas_positivas(df):
    return df[df['Unidades'] > 0]

# Define a function to calculate sold volume for each product
def calcular_volumen_vendido(df, productos):
    productos['SnkProducto'] = productos['SnkProducto'].astype(str)
    merged_df = df.merge(productos, on="SnkProducto", how="inner")
    return merged_df

# Define a function to process dates in sales data
def procesar_fechas(df):
    df['SnkFecha'] = pd.to_datetime(df['SnkFecha'], format="%Y%m%d") - pd.to_timedelta(7, unit='d')
    df = df.groupby([pd.Grouper(key='SnkFecha', freq='W'), 'Proyecto', 'SnkProducto'])['Unidades'].sum().reset_index()
    return df

# Define a function to generate combinations of dates, products, and cities
def generar_combinaciones(df):
    fechas = df['SnkFecha'].unique()
    productos = df['SnkProducto'].unique()
    ciudades = df['Proyecto'].unique()
    combinaciones = list(product(fechas, ciudades, productos))
    combinaciones_df = pd.DataFrame(combinaciones, columns=['SnkFecha', 'Proyecto', 'SnkProducto'])
    return df, combinaciones_df

# Define a function to combine dataframes
def combinar_dataframes(df, combinaciones_df):
    return combinaciones_df.merge(df, on=['SnkFecha', 'Proyecto', 'SnkProducto'], how='left')

# Define a function to fill null values in sales data
def llenar_valores_nulos(df):
    df['Unidades'].fillna(0, inplace=True)
    return df

# Define a function to add a 'Pais' category to sales data
def agregar_categoria_pais(df):
    df['Pais'] = "Colombia"
    return df

# Define a function for preprocessing sales data
def preprocessing(Ventas1, productos):
    productos = filter_producto(productos)
    steps = [
        ('subset_ventas', FunctionTransformer(subset_ventas)),
        ('eliminar_duplicados', FunctionTransformer(eliminar_duplicados)),
        ('seleccionar_columnas', FunctionTransformer(seleccionar_columnas)),
        ('filtrar_ventas_positivas', FunctionTransformer(filtrar_ventas_positivas)),
        ('calcular_volumen_vendido', FunctionTransformer(calcular_volumen_vendido, kw_args={'productos': productos})),
        ('procesar_fechas', FunctionTransformer(procesar_fechas)),
        ('generar_combinaciones', FunctionTransformer(generar_combinaciones))
      ]
    pipeline_1 = Pipeline(steps)
    Final, combinaciones_df = pipeline_1.fit_transform(Ventas1)
    # Aplicar las funciones en orden para crear el pipeline
    steps_2 = [
        ('combinar_dataframes', FunctionTransformer(combinar_dataframes, kw_args={'combinaciones_df': combinaciones_df})),  # Use custom transformer
        ('llenar_valores_nulos', FunctionTransformer(llenar_valores_nulos)),
        ('agregar_categoria_pais', FunctionTransformer(agregar_categoria_pais)),
    ]

    pipeline_2 = Pipeline(steps_2)

    # Apply the pipeline to the Ventas1 dataframe
    Ventas1_processed = pipeline_2.fit_transform(Final)


    Final_vol, H_vol, tags_vol = aggregate_hierarchies(Ventas1_processed, 'Unidades')
    return Final_vol, H_vol, tags_vol

# Define a function to split data for volume forecasting
def split_data_vol(resultado_vol):
    test = resultado_vol.groupby('unique_id').tail(26)
    lbl = prep.LabelEncoder()
    test['unique_id'] = lbl.fit_transform(test['unique_id'].astype(str))
    test['ds'] = lbl.fit_transform(test['ds'].astype(str))

    train = resultado_vol.drop(test.index)
    train['unique_id'] = lbl.fit_transform(train['unique_id'].astype(str))
    train['ds'] = lbl.fit_transform(train['ds'].astype(str))

    return train, test

def objective(space):
    clf=XGBRegressor(
                    n_estimators =space['n_estimators'],
                    max_depth = int(space['max_depth']),
                    gamma = space['gamma'],
                    reg_alpha = int(space['reg_alpha']),
                    min_child_weight=int(space['min_child_weight']),
                    colsample_bytree=int(space['colsample_bytree']),
                    eval_metric=mean_absolute_error)

    evaluation = [ ( test_vol.drop(columns=['y']), test_vol.y)]

    clf.fit(train_vol.drop(columns=['y']), train_vol.y,
            eval_set=evaluation,
            early_stopping_rounds=10,verbose=False)


    pred = clf.predict(test_vol.drop(columns=['y']))
    RMSE = rmse(test_vol.y, pred)
    print ("RMSE:", RMSE)
    return {'loss': RMSE, 'status': STATUS_OK }

def cross_validation_sf(resultado_vol, seasons, types):
    sf_Vol = {}
    for i in seasons:
        print(i)
        for k in types:
            print(k)
            sf_Volumen = StatsForecast(
                df=resultado_vol,
                models=[DOT(season_length=i, decomposition_type=k)],
                freq='W',
                n_jobs=-1
            )
            cross_volumen = sf_Volumen.cross_validation(
                df=resultado_vol,
                h=4,
                step_size=4,
                n_windows=12
            )
            sf_Vol[rmse(cross_volumen['y'], cross_volumen['DynamicOptimizedTheta'])] = [i, k]

    return sf_Vol
def optimize_hyperparameters(space):
    trials = Trials()

    best_hyperparams = fmin(fn = objective,
                        space = space,
                        algo = tpe.suggest,
                        max_evals = 100,
                        trials = trials)

    return best_hyperparams
def split_data(data):
    test_data = data.groupby('unique_id').tail(8)
    train_data = data.drop(test_data.index)
    test_data = test_data.set_index('unique_id')
    train_data = train_data.set_index('unique_id')
    return train_data, test_data

def create_stats_forecast(train_data, sf_params):
    sf_object = StatsForecast(
        df=train_data,
        models=[DOT(season_length=sf_params[min(sf_params)][0], decomposition_type=sf_params[min(sf_params)][1])],
        freq='W',
        n_jobs=-1
    )
    return sf_object

def create_ml_forecast(df, ml_params):
    mlf = MLForecast(
        models=[XGBRegressor(
            max_depth=int(ml_params['max_depth']),
            gamma=ml_params['gamma'],
            reg_alpha=ml_params['reg_alpha'],
            min_child_weight=ml_params['min_child_weight'],
            colsample_bytree=ml_params['colsample_bytree'],
            reg_lambda=ml_params['reg_alpha']
        )],
        freq='W',
        lags=list(range(1, 53))
    )
    return mlf

def create_neural_forecast(df):
    nf = NeuralForecast(
        models=[AutoLSTM(h=74, config=None, backend='optuna')],
        freq='W'
    )
    return nf

def fit_ml_forecast(mlf, train_data, ho):
    fit_ml = mlf.fit(train_data.reset_index())
    forecasts_ml = fit_ml.predict(h=ho).set_index('unique_id')
    return mlf, forecasts_ml

def fit_neural_forecast(nf, train_data):
    nf.fit(df=train_data.reset_index())
    forecast_nn = nf.predict()
    forecast_nn_test = forecast_nn.groupby('unique_id').head(8)
    return forecast_nn, forecast_nn_test

def forecast_stats_volume(sf_object, ho):
    forecasts_stats = sf_object.forecast(h=ho)
    return sf_object, forecasts_stats

def merge_forecasts(stats_forecast, ml_forecast, neural_forecast):
    forecasts_volume = stats_forecast.merge(ml_forecast, on=['unique_id', 'ds']).merge(neural_forecast, on=['unique_id', 'ds'])
    return forecasts_volume

def reconcile_forecasts(forecasts_volume, train_data, h_volume, tags_volume):
    hrec = HierarchicalReconciliation(
        reconcilers=[
            MinTrace(method='ols', nonnegative=True),
            OptimalCombination(method='wls_struct', nonnegative=True),
            BottomUp()
        ]
    )
    forecasts_rec_volume = hrec.reconcile(Y_hat_df=forecasts_volume, Y_df=train_data, S=h_volume, tags=tags_volume)
    return forecasts_rec_volume

def mean_squared_scaled_error(y, y_hat):
    mse = np.mean((y - y_hat) ** 2)
    variance = np.var(y)
    msse = round(mse / variance, 3)
    return msse

def cumulative_accuracy(y, y_hat):
    sum_predictions = np.sum(y_hat)
    sum_actual = np.sum(y)
    if sum_predictions > sum_actual:
        accuracy = round((sum_actual / sum_predictions) * 100, 2)
    else:
        accuracy = round((sum_predictions / sum_actual) * 100, 2)
    return accuracy

def mean_squared_error(y, y_hat):
    mse_value = round(np.mean((np.array(y) - np.array(y_hat)) ** 2), 2)
    return mse_value

def hierarchical_evaluation(forecasts_rec_volume, test_data, tags_volume):
    evaluator = HierarchicalEvaluation(evaluators=[mean_squared_scaled_error, cumulative_accuracy, mean_squared_error])
    vol_eval = evaluator.evaluate(Y_hat_df=forecasts_rec_volume, Y_test_df=test_data, tags=tags_volume).astype(float)
    msse_rows = vol_eval.index.get_level_values('metric').isin(['mean_squared_scaled_error', 'mean_squared_error'])
    c_acc_rows = ~msse_rows
    vol_eval['best_model'] = np.where(msse_rows, vol_eval.apply(lambda row: row.idxmin(), axis=1),
                                      vol_eval.apply(lambda row: row.idxmax(), axis=1))
    return vol_eval
def save_and_download_evaluation(vol_eval, filename='eval_vol.csv'):
    vol_eval.to_csv(filename, encoding='utf-8-sig')
    files.download(filename)
    return vol_eval
def evaluate_model(resultado_vol, best_hyperparams, H_vol, tags_vol, sf_Vol):
    print("fiting models")
    train_volume, test_volume = split_data(resultado_vol)
    steps = [
        ('create_stats_forecast', FunctionTransformer(create_stats_forecast, kw_args={'sf_params': sf_Vol})),  # Use custom transformer
        ('forecast_stats_volume', FunctionTransformer(forecast_stats_volume, kw_args={'ho': 8}))
      ]
    pipeline_1 = Pipeline(steps)
    sf_volume, forecasts_stats_volume = pipeline_1.fit_transform(train_volume)
    # Aplicar las funciones en orden para crear el pipeline
    steps_1 = [
        ('create_ml_forecast', FunctionTransformer(create_ml_forecast, kw_args={'ml_params':best_hyperparams })),  # Use custom transformer
        ('fit_ml_forecast', FunctionTransformer(fit_ml_forecast, kw_args={'train_data':train_volume , 'ho': 8}))
      ]
    pipeline_2 = Pipeline(steps_1)
    ml_forecast, forecasts_ml_volume = pipeline_2.fit_transform(train_volume)
    # Aplicar las funciones en orden para crear el pipeline
    steps_2 = [
        ('create_neural_forecast', FunctionTransformer(create_neural_forecast)),  # Use custom transformer
        ('fit_neural_forecast', FunctionTransformer(fit_neural_forecast, kw_args={'train_data':train_volume}))
      ]
    pipeline_3 = Pipeline(steps_2)
    forecast_nn_volume, forecast_nn_test_volume = pipeline_3.fit_transform(train_volume)
    print("hierarchical compensation and evaluation")
    steps_3 = [
        ('create_neural_forecast', FunctionTransformer(merge_forecasts, kw_args={'ml_forecast':forecasts_ml_volume, 'neural_forecast': forecast_nn_test_volume})),  # Use custom transformer
        ('reconcile_forecasts', FunctionTransformer(reconcile_forecasts, kw_args={'train_data':train_volume, 'h_volume':H_vol, 'tags_volume': tags_vol})),
        ('hierarchical_evaluation', FunctionTransformer(hierarchical_evaluation, kw_args={'test_data':test_volume, 'tags_volume':tags_vol})),  # Use custom transformer
        ('save_and_download_evaluation', FunctionTransformer(save_and_download_evaluation))
      ]
    pipeline_4 = Pipeline(steps_3)
    evaluation_volume = pipeline_4.fit_transform(forecasts_stats_volume)
    return forecast_nn_volume, ml_forecast, sf_volume, train_volume, evaluation_volume

def process_best_models(vol_eval, tags_vol):
    vol_eval.reset_index(inplace=True)
    best_model_acc_vol = vol_eval[vol_eval['metric'] == 'cumulative_accuracy'][['level', 'best_model']].set_index('level').to_dict()['best_model']
    best_model_acc_vol['Pais'] = best_model_acc_vol['Overall']
    best_model_acc_vol['Pais/Proyecto'] = best_model_acc_vol['Overall']
    best_model_acc_vol['Pais/Proyecto/SnkProducto'] = best_model_acc_vol['Overall']
    best_model_acc_vol.pop('Overall')

    return best_model_acc_vol
def reconcile_forecasts_and_post_process(forecasts_volumen, train_Vol, H_vol, tags_vol):
    hrec = HierarchicalReconciliation(
        reconcilers=[
            MinTrace(method='ols', nonnegative=True),
            OptimalCombination(method='wls_struct', nonnegative=True),
            BottomUp()
        ]
    )
    forecasts_rec_volumen = hrec.reconcile(Y_hat_df=forecasts_volumen, Y_df=train_Vol, S=H_vol, tags=tags_vol)

    final_vol = pd.DataFrame(columns=['unique_id', 'ds', 'prediction'])
    forecasts_rec_volumen.reset_index(inplace=True)

    for key in tags_vol.keys():
        copi1 = forecasts_rec_volumen[forecasts_rec_volumen['unique_id'].isin(tags_vol[key])][['unique_id', 'ds', best_model_acc_vol[key]]]
        copi1.rename(columns={best_model_acc_vol[key]: 'prediction'}, inplace=True)
        final_vol = final_vol.append(copi1, ignore_index=True)

    final_vol['ds'] = pd.to_datetime(final_vol['ds'])

    # Group by 'unique_id' and 'ds' (year-month) and apply different aggregation functions
    final_vol = final_vol.groupby(['unique_id', final_vol['ds'].dt.to_period("M")]).agg({
        'prediction': 'sum'
    }).reset_index()

    final_vol['prediction'] = round(final_vol['prediction'], 0).astype(int)

    return final_vol
def process_and_download_predictions(final_vol, resultado_vol, tags_vol, productos):
    resultado_vol['ds'] = pd.to_datetime(resultado_vol['ds'])

    # Group by 'unique_id' and 'ds' (year-month) and apply different aggregation functions
    resultado_vol = resultado_vol.groupby(['unique_id', resultado_vol['ds'].dt.to_period("M")]).agg({
        'y': 'sum'
    }).reset_index()
    resultado_vol.rename(columns={'y': 'real'}, inplace=True)
    resultado_vol['real'] = round(resultado_vol['real'], 0).astype(int)

    for key in tags_vol.keys():
        copi1 = resultado_vol[resultado_vol['unique_id'].isin(tags_vol[key])]
        b_vol = final_vol[final_vol['unique_id'].isin(tags_vol[key])]
        merge_vol = pd.merge(copi1, b_vol, on=['unique_id', 'ds'], how='outer')
        merge_vol = merge_vol.sort_values(['unique_id', 'ds'])
        merge_vol[['Año', 'Mes']] = merge_vol['ds'].astype(str).str.split('-', expand=True)
        merge_vol.drop(columns=['ds'], inplace=True)

        if key == 'Pais/Proyecto':
            merge_vol[['Pais', 'Ciudad']] = merge_vol['unique_id'].str.split('/', expand=True)
            merge_vol.drop(columns=['unique_id'], inplace=True)
        elif key == 'Pais/Proyecto/SnkProducto':
            merge_vol[['Pais', 'Ciudad', 'Producto']] = merge_vol['unique_id'].str.split('/', expand=True)
            productos['SnkProducto'] = productos['SnkProducto'].astype(str)
            merge_vol = merge_vol.merge(productos[['ClaseContable', 'SnkProducto', 'Volumen']],
                                        left_on='Producto', right_on='SnkProducto', how='left')
            merge_vol.rename(columns={'ClaseContable': 'Categoria', 'prediction': 'Pronostico'}, inplace=True)
            merge_vol.drop(columns=['unique_id', 'SnkProducto', 'Volumen'], inplace=True)
        else:
            merge_vol.rename(columns={'unique_id': 'Pais', 'prediction': 'Pronostico'}, inplace=True)

        merge_vol.to_csv(f'predicciones_vol_{key.replace("/", "-")}.csv', encoding='utf-8-sig')
        files.download(f'predicciones_vol_{key.replace("/", "-")}.csv')

print("preprocesamiento")
resultado_vol, H_vol, tags_vol = preprocessing(Ventas, productos)
resultado_vol.drop(resultado_vol.groupby('unique_id').tail(3).index, inplace = True)
print("hyperparameter tunning (DOT)")
seasons = [1, 13, 26, 52]
types = ['multiplicative', 'additive']

train_vol, test_vol = split_data_vol(resultado_vol)
sf_Vol = cross_validation_sf(resultado_vol, seasons, types)

print("XGBOOST")
space = {'max_depth': hp.quniform("max_depth", 3, 18, 1),
         'gamma': hp.uniform('gamma', 1, 9),
         'reg_alpha': hp.quniform('reg_alpha', 40, 180, 1),
         'reg_lambda': hp.uniform('reg_lambda', 0, 1),
         'colsample_bytree': hp.uniform('colsample_bytree', 0.5, 1),
         'min_child_weight': hp.quniform('min_child_weight', 0, 10, 1),
         'n_estimators': 180,
         'seed': 0}

best_hyperparams = optimize_hyperparameters(space)
forecast_nn_volume, ml_forecast, sf_volume, train_volume, evaluation_volume = evaluate_model(resultado_vol, best_hyperparams, H_vol, tags_vol, sf_Vol)

print("making forecast with best model")
best_model_acc_vol = process_best_models(evaluation_volume, tags_vol)
_, forecasts_ml_volume = fit_ml_forecast(ml_forecast, train_volume, 74)
_, forecasts_stats_volume = forecast_stats_volume(sf_volume, 74)
forecasts_volume = merge_forecasts(forecasts_stats_volume, forecasts_ml_volume, forecast_nn_volume)
print("post process")
final_vol = reconcile_forecasts_and_post_process(forecasts_volume, train_volume, H_vol, tags_vol)
process_and_download_predictions(final_vol, resultado_vol, tags_vol, productos)


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.1/45.1 kB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 452.8/452.8 kB 15.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 357.5/357.5 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 381.6/381.6 kB 40.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.7/102.7 kB 16.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 679.5/679.5 kB 36.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.5/296.5 kB 30.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/4.7 MB 73.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 84.4 MB/s eta 0:00:00
  Created wheel for htmlmin: filename=htmlmin-0.1.12-py3-none-any.whl size=27082 sha256=bcb7545915002fe263b83ab28c0e5f2d12286d159e192149a4c2b1e2f656a54e
  Stored in directory: /root/.cache/pip/wheels/dd/91/29/a79cecb328d0173

/usr/local/lib/python3.10/dist-packages/statsforecast/core.py:25: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


Mounted at /content/drive
preprocesamiento
hyperparameter tunning (DOT)
1
multiplicative
additive
13
multiplicative
additive
26
multiplicative
additive
52
multiplicative
additive
XGBOOST
RMSE:
3520.8569204171804
RMSE:
3517.6987444561146
RMSE:
3517.698529897132
RMSE:
3520.856492772302
RMSE:
3520.852385291207
RMSE:
3520.856137750911
RMSE:
3517.69898940344
RMSE:
3524.1409602767726
RMSE:
3517.70310618357
RMSE:
3527.3029358840613
RMSE:
3520.851522143528
RMSE:
3517.4375343306556
RMSE:
3517.509500824896
RMSE:
3527.3047480298783
RMSE:
3525.7817007782837
RMSE:
3517.8972836944604
RMSE:
3517.6478208654826
RMSE:
3517.437512978079
RMSE:
3517.4895342535297
RMSE:
3517.702468831297
RMSE:
3517.4374833121847
RMSE:
3517.4374833121847
RMSE:
3517.4357772012645
RMSE:
3517.4358043552324
RMSE:
3517.4357772012645
RMSE:
3517.4361410500123
RMSE:
3517.437545746237
RMSE:
3517.435882664529
RMSE:
3517.437545746237
RMSE:
3517.4375604684365
RMSE:
3517.436257521638
RMSE:
3517.4375091487655
RMSE:
3517.4373192263606
RMSE

[I 2023-11-22 01:35:00,386] A new study created in memory with name: no-name-0cf530b6-4241-4753-9fcb-78580f6e5c6a
INFO:lightning_fabric.utilities.seed:Seed set to 6


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

...

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>